In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report

import random
import numpy as np
import torch

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If you are using CUDA
torch.backends.cudnn.deterministic = True  # For deterministic results
torch.backends.cudnn.benchmark = False  # For consistency across different environments


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMAGE_DIR = 'D:\\PAD-UFES\\images'  
METADATA_PATH = 'D:\\PAD-UFES\\metadata.csv'

metadata = pd.read_csv(METADATA_PATH)


def preprocess_metadata(metadata):
    metadata = metadata.fillna('UNK')

    boolean_cols = [
        'smoke',
        'drink',
        'pesticide',
        'skin_cancer_history',
        'cancer_history',
        'has_piped_water',
        'has_sewage_system',
        'itch',
        'grew',
        'hurt',
        'changed',
        'bleed',
        'elevation',
        'biopsed',
    ]
    # Ensure columns are strings and lowercase
    for col in boolean_cols:
        metadata[col] = metadata[col].astype(str).str.lower()
    
    # Map boolean columns to 1/0/-1
    boolean_mapping = {'true': 1, 'false': 0, 'unk': -1}
    for col in boolean_cols:
        metadata[col] = metadata[col].map(boolean_mapping)
    
    # Handle categorical variables
    categorical_cols = [
        'background_father',
        'background_mother',
        'gender',
        'region',
        'diagnostic',
    ]
    # Convert categorical columns to string
    for col in categorical_cols:
        metadata[col] = metadata[col].astype(str)
    
    # One-hot encode categorical variables
    metadata_encoded = pd.get_dummies(metadata[categorical_cols])
    
    # Normalize numerical variables
    numerical_cols = ['age', 'fitspatrick', 'diameter_1', 'diameter_2']
    # Ensure numerical columns are numeric
    for col in numerical_cols:
        metadata[col] = pd.to_numeric(metadata[col], errors='coerce')
    # Fill NaNs in numerical columns with the mean
    metadata[numerical_cols] = metadata[numerical_cols].fillna(metadata[numerical_cols].mean())
    # Scale numerical columns
    scaler = StandardScaler()
    metadata_numeric = metadata[numerical_cols]
    metadata_numeric_scaled = pd.DataFrame(
        scaler.fit_transform(metadata_numeric), columns=numerical_cols
    )
    
    # Combine all metadata features
    metadata_processed = pd.concat(
        [metadata_numeric_scaled.reset_index(drop=True),
         metadata_encoded.reset_index(drop=True),
         metadata[boolean_cols].reset_index(drop=True)], axis=1
    )
    
    return metadata_processed

# Preprocess metadata
metadata_processed = preprocess_metadata(metadata)

def get_image_paths(metadata, image_dir):
    image_paths = []
    for idx, row in metadata.iterrows():
        filename = row['img_id']
        # Ensure filename is a string
        filename = str(filename)
        # Check if filename has an extension
        if not filename.endswith(('.jpg', '.jpeg', '.png')):
            # Try common extensions
            possible_extensions = ['.jpg', '.jpeg', '.png']
            found = False
            for ext in possible_extensions:
                filepath = os.path.join(image_dir, filename + ext)
                if os.path.isfile(filepath):
                    image_paths.append(filepath)
                    found = True
                    break
            if not found:
                print(f"Image file not found for ID: {filename}")
                image_paths.append(None)
        else:
            filepath = os.path.join(image_dir, filename)
            if os.path.isfile(filepath):
                image_paths.append(filepath)
            else:
                print(f"Image file not found: {filepath}")
                image_paths.append(None)
    metadata['ImagePath'] = image_paths
    return metadata

metadata = get_image_paths(metadata, IMAGE_DIR)

# Remove entries with missing images
metadata = metadata[metadata['ImagePath'].notnull()]
metadata_processed = metadata_processed.loc[metadata.index].reset_index(drop=True)
metadata = metadata.reset_index(drop=True)

# Drop diagnostic-related columns from features
diagnostic_cols = ['diagnostic_ACK', 'diagnostic_BCC', 'diagnostic_MEL', 'diagnostic_NEV', 'diagnostic_SCC', 'diagnostic_SEK']
metadata_processed = metadata_processed.drop(columns=diagnostic_cols)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(metadata['diagnostic'])
num_classes = len(label_encoder.classes_)

# Split data into features and labels
X_meta = metadata_processed.reset_index(drop=True)
X_img_paths = metadata['ImagePath'].reset_index(drop=True)
y = pd.Series(y_encoded)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Count original labels
original_counts = Counter(y)
class_labels = label_encoder.classes_
class_indices = list(range(len(class_labels)))
counts = [original_counts[i] for i in class_indices]

# Print counts
print("📊 Original Class Distribution:")
for cls, count in zip(class_labels, counts):
    print(f"{cls:<10} → {count} samples")

# Plot
plt.figure(figsize=(8, 5))
sns.barplot(x=class_labels, y=counts)
plt.title("Original Class Distribution")
plt.ylabel("Number of Samples")
plt.xlabel("Class")
plt.tight_layout()
plt.show()

In [ ]:
from collections import Counter
counts = Counter(y)
ir = max(counts.values()) / min(counts.values())
print(f'Imbalance Ratio (IR): {ir:.2f}')

In [ ]:
def gini(array):
    array = np.sort(np.array(array))
    index = np.arange(1, array.shape[0]+1)
    return (2 * np.sum(index * array)) / (array.sum() * array.shape[0]) - (array.shape[0] + 1) / array.shape[0]

gini_value = gini(list(Counter(y).values()))
print(f'Gini Imbalance Score: {gini_value:.4f}')

In [ ]:
import os
import shutil
from PIL import Image
from torchvision import transforms
import numpy as np
import pandas as pd
from collections import Counter
import random
from sklearn.preprocessing import LabelEncoder
from imblearn.combine import SMOTETomek
import imagehash

# === Configuration ===
AUGMENTED_DIR = 'D:/PAD-UFES/augmented_images_balanced_800'

# Recreate output directory
if os.path.exists(AUGMENTED_DIR):
    shutil.rmtree(AUGMENTED_DIR)
os.makedirs(AUGMENTED_DIR, exist_ok=True)

# === Augmentation Transform for saving (NO Normalize or ToTensor) ===
offline_aug_transform_save = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomApply([
        transforms.ColorJitter(0.3, 0.3, 0.3, 0.1),
        transforms.RandomRotation(45),
        transforms.RandomHorizontalFlip(),
    ], p=1.0),  # Force transformation
])

# === Label Encoding ===
label_encoder = LabelEncoder()
metadata['diagnostic'] = metadata['diagnostic'].astype(str)
y_all = label_encoder.fit_transform(metadata['diagnostic'])
metadata['EncodedLabel'] = y_all
metadata_processed = metadata_processed.reset_index(drop=True)

# === Ensure metadata is numeric for SMOTETomek ===
metadata_processed = metadata_processed.apply(pd.to_numeric, errors='coerce').fillna(0)

# === SMOTETomek Augmentation on Metadata ===
print("\n⚙️ Applying SMOTETomek on metadata...")
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(metadata_processed, y_all)

# Identify synthetic samples
original_len = len(metadata_processed)
X_synthetic = X_resampled[original_len:]
y_synthetic = y_resampled[original_len:]

X_synthetic = np.array(X_synthetic)

# Group real image paths by label
class_to_img_paths = metadata.groupby('EncodedLabel')['ImagePath'].apply(list).to_dict()

# === Initialize augmented data holders ===
augmented_metadata = []
augmented_img_paths = []
augmented_labels = []
image_hashes = []  # for perceptual hashes
meta_set = set()

# === Generate Augmented Image + Metadata ===
print(f"\n🎨 Creating {len(X_synthetic)} synthetic image–metadata pairs...")
for i, (meta_row, label_id) in enumerate(zip(X_synthetic, y_synthetic)):
    meta_row = np.array(meta_row, dtype=np.float64)
    meta_tuple = tuple(np.round(meta_row, 5))
    if meta_tuple in meta_set:
        continue  # Skip duplicate metadata

    # Randomly pick an image from the same class
    img_candidates = class_to_img_paths.get(label_id, [])
    if not img_candidates:
        continue

    chosen_img_path = random.choice(img_candidates)
    try:
        image = Image.open(chosen_img_path).convert('RGB')
    except:
        print(f"⚠️ Failed to load: {chosen_img_path}")
        continue

    aug_img = offline_aug_transform_save(image)
    img_hash = imagehash.phash(aug_img)

    if any(img_hash - h <= 5 for h in image_hashes):
        continue  # Skip visually similar image

    aug_filename = f"aug_{label_id}_{i}.jpg"
    aug_path = os.path.join(AUGMENTED_DIR, aug_filename)
    aug_img.save(aug_path, format='JPEG', quality=95)

    image_hashes.append(img_hash)
    meta_set.add(meta_tuple)
    augmented_img_paths.append(aug_path)
    augmented_metadata.append(meta_row)
    augmented_labels.append(label_id)

# === Combine into DataFrames ===
aug_meta_df = pd.DataFrame(augmented_metadata, columns=metadata_processed.columns)
aug_img_paths_series = pd.Series(augmented_img_paths)
aug_labels_series = pd.Series(augmented_labels)

print(f"\n✅ Finished generating {len(aug_labels_series)} clean synthetic samples.")
print(f"Images saved to: '{AUGMENTED_DIR}'")

# Save outputs
augmented_df = pd.DataFrame({
    'ImagePath': aug_img_paths_series,
    'Label': aug_labels_series
})
aug_meta_df.to_csv("D:/PAD-UFES/augmented_metadata.csv", index=False)
augmented_df.to_csv("D:/PAD-UFES/augmented_labels.csv", index=False)

# Debug summary
from collections import Counter
print("\n📊 Synthetic class distribution:")
print(Counter(aug_labels_series))
print("✅ No duplicate metadata rows?", aug_meta_df.duplicated().sum() == 0)

In [ ]:
from sklearn.model_selection import train_test_split

# First: Train + Temp
X_train_meta, X_temp_meta, X_train_img_paths, X_temp_img_paths, y_train, y_temp = train_test_split(
    X_meta,
    X_img_paths,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# Then: Temp → Validation + Test
X_val_meta, X_test_meta, X_val_img_paths, X_test_img_paths, y_val, y_test = train_test_split(
    X_temp_meta,
    X_temp_img_paths,
    y_temp,
    test_size=0.5,
    stratify=y_temp,
    random_state=42
)

# Load augmented metadata + image paths
aug_meta_df = pd.read_csv("D:/PAD-UFES/augmented_metadata.csv")
aug_labels_df = pd.read_csv("D:/PAD-UFES/augmented_labels.csv")

# Combine augmented samples with training set
X_train_meta_final = pd.concat([X_train_meta, aug_meta_df], ignore_index=True)
X_train_img_paths_final = pd.concat([X_train_img_paths.reset_index(drop=True),
                                     aug_labels_df['ImagePath']], ignore_index=True)
y_train_final = pd.concat([y_train.reset_index(drop=True),
                           aug_labels_df['Label']], ignore_index=True)


In [ ]:
import os
import pandas as pd
from PIL import Image
import imagehash

# === Paths ===
AUG_IMG_DIR = "D:/PAD-UFES/augmented_images_balanced_800"
AUG_META_CSV = "D:/PAD-UFES/augmented_metadata.csv"
AUG_LABELS_CSV = "D:/PAD-UFES/augmented_labels.csv"

# Load augmented CSVs
aug_meta_df = pd.read_csv(AUG_META_CSV)
aug_labels_df = pd.read_csv(AUG_LABELS_CSV)

# Extract augmented image paths
aug_img_paths = aug_labels_df['ImagePath'].tolist()

# Validation and test paths (define these if not already available)
val_paths = X_val_img_paths.tolist()
test_paths = X_test_img_paths.tolist()

# === Compute perceptual hashes ===
def compute_hashes(paths):
    hash_dict = {}
    for p in paths:
        try:
            img = Image.open(p).resize((224, 224)).convert('RGB')
            img_hash = imagehash.phash(img)
            hash_dict[p] = img_hash
        except:
            continue
    return hash_dict

val_hashes = compute_hashes(val_paths)
test_hashes = compute_hashes(test_paths)
aug_hashes = compute_hashes(aug_img_paths)

# === Find and delete leaks ===
hash_threshold = 5
to_delete = []

for aug_path, aug_hash in aug_hashes.items():
    for ref_hash in list(val_hashes.values()) + list(test_hashes.values()):
        if aug_hash - ref_hash <= hash_threshold:
            to_delete.append(aug_path)
            break  # no need to check further if one match is found

print(f"\n🧹 Found {len(to_delete)} augmented images to delete due to potential leakage.")

# === Delete images and remove entries from CSVs ===
aug_labels_df_clean = aug_labels_df[~aug_labels_df['ImagePath'].isin(to_delete)].reset_index(drop=True)
aug_meta_df_clean = aug_meta_df.loc[aug_labels_df_clean.index].reset_index(drop=True)

for img_path in to_delete:
    try:
        os.remove(img_path)
    except:
        print(f"⚠️ Couldn't delete: {img_path}")

# Save cleaned CSVs
aug_labels_df_clean.to_csv(AUG_LABELS_CSV, index=False)
aug_meta_df_clean.to_csv(AUG_META_CSV, index=False)

print("✅ Cleaned CSVs and removed leaked images.")

In [ ]:
import matplotlib.pyplot as plt

# === Separate distances ===
val_dists = []
test_dists = []

for aug_hash in aug_hashes.values():
    for ref_hash in val_hashes.values():
        val_dists.append(aug_hash - ref_hash)
    for ref_hash in test_hashes.values():
        test_dists.append(aug_hash - ref_hash)

# === Plotting ===
plt.figure(figsize=(8, 5))
plt.hist(val_dists, bins=30, alpha=0.6, label='Augmented vs Validation', color='blue', edgecolor='black')
plt.hist(test_dists, bins=30, alpha=0.6, label='Augmented vs Test', color='green', edgecolor='black')
plt.axvline(x=5, color='red', linestyle='--', label='Threshold (5)')

plt.title("Hamming Distance Between Augmented and Val/Test Sets")
plt.xlabel("Hamming Distance")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style and color palette
plt.style.use('seaborn-v0_8-paper')
sns.set_palette("deep")

# Create the figure and axes
plt.figure(figsize=(10, 6))

# Plot histograms with enhanced aesthetics
sns.histplot(val_dists, bins=30, alpha=0.7, label='Augmented vs Validation', 
             color=sns.color_palette("deep")[0], 
             edgecolor='white', 
             linewidth=1)
sns.histplot(test_dists, bins=30, alpha=0.7, label='Augmented vs Test', 
             color=sns.color_palette("deep")[1], 
             edgecolor='white', 
             linewidth=1)

# Add vertical line with improved styling
plt.axvline(x=5, color='red', linestyle='--', linewidth=2, label='Threshold (5)')

# Enhance title and labels
plt.title("Hamming Distance Distribution", fontsize=16, fontweight='bold')
plt.xlabel("Hamming Distance", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# Improve legend
plt.legend(frameon=True, framealpha=0.8, facecolor='white', edgecolor='gray')

# Add grid with subtler styling
plt.grid(True, linestyle='--', linewidth=0.5, color='gray', alpha=0.7)

# Adjust layout and save
plt.tight_layout()
plt.show()

In [ ]:
# Save validation and test metadata for future distance checks
X_val_meta.to_csv("D:/PAD-UFES/val_metadata.csv", index=False)
X_test_meta.to_csv("D:/PAD-UFES/test_metadata.csv", index=False)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === Load metadata ===
aug_meta_df = pd.read_csv("D:/PAD-UFES/augmented_metadata.csv")
val_meta_df = pd.read_csv("D:/PAD-UFES/val_metadata.csv")     # Save this when splitting
test_meta_df = pd.read_csv("D:/PAD-UFES/test_metadata.csv")   # Save this when splitting

# Convert to numpy arrays
X_aug = aug_meta_df.values
X_val = val_meta_df.values
X_test = test_meta_df.values

# === 1. Euclidean Distance (Val & Test) ===
eucl_val = euclidean_distances(X_aug, X_val).min(axis=1)
eucl_test = euclidean_distances(X_aug, X_test).min(axis=1)

# === 2. Cosine Distance (1 - similarity) ===
cos_val = 1 - cosine_similarity(X_aug, X_val).max(axis=1)
cos_test = 1 - cosine_similarity(X_aug, X_test).max(axis=1)

# === Plotting ===
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(eucl_val, bins=30, alpha=0.7, label="Aug vs Val", color='teal', edgecolor='black')
plt.hist(eucl_test, bins=30, alpha=0.7, label="Aug vs Test", color='purple', edgecolor='black')
plt.axvline(x=0.3, color='red', linestyle='--', label='Threshold')
plt.title("Euclidean Distance: Augmented → Val/Test")
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(cos_val, bins=30, alpha=0.7, label="Aug vs Val", color='orange', edgecolor='black')
plt.hist(cos_test, bins=30, alpha=0.7, label="Aug vs Test", color='gray', edgecolor='black')
plt.axvline(x=0.1, color='red', linestyle='--', label='Threshold')
plt.title("Cosine Distance: Augmented → Val/Test")
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

# === Load metadata ===
aug_meta_df = pd.read_csv("D:/PAD-UFES/augmented_metadata.csv")
val_meta_df = pd.read_csv("D:/PAD-UFES/val_metadata.csv")
test_meta_df = pd.read_csv("D:/PAD-UFES/test_metadata.csv")

# === Convert to arrays ===
X_aug = aug_meta_df.values
X_val = val_meta_df.values
X_test = test_meta_df.values

# === Euclidean Distance ===
eucl_val_matrix = euclidean_distances(X_aug, X_val)
eucl_test_matrix = euclidean_distances(X_aug, X_test)
eucl_val_dists = eucl_val_matrix.min(axis=1)
eucl_test_dists = eucl_test_matrix.min(axis=1)

# === Cosine Distance ===
cos_val_sim = cosine_similarity(X_aug, X_val)
cos_test_sim = cosine_similarity(X_aug, X_test)
cos_val_dists = 1 - cos_val_sim.max(axis=1)
cos_test_dists = 1 - cos_test_sim.max(axis=1)

# === Summary function ===
def summarize_distances(name, distances, threshold):
    return {
        'Type': name,
        'Mean': round(np.mean(distances), 4),
        'Min': round(np.min(distances), 4),
        'Max': round(np.max(distances), 4),
        'Samples Below Threshold': int(np.sum(distances < threshold))
    }

# === Thresholds ===
eucl_thresh = 0.3
cos_thresh = 0.1

# === Collect statistics ===
summary_data = [
    summarize_distances("Euclidean (Aug vs Val)", eucl_val_dists, eucl_thresh),
    summarize_distances("Euclidean (Aug vs Test)", eucl_test_dists, eucl_thresh),
    summarize_distances("Cosine (Aug vs Val)", cos_val_dists, cos_thresh),
    summarize_distances("Cosine (Aug vs Test)", cos_test_dists, cos_thresh),
]

# === Create and display DataFrame ===
summary_df = pd.DataFrame(summary_data)
print("\n📊 Distance Summary Table:")
print(summary_df)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === Style ===
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# === Euclidean Distance Plot ===
plt.figure(figsize=(10, 7))
plt.hist(eucl_val, bins=30, alpha=0.7, label="Augmented vs Validation", color='#1E88E5', edgecolor='white')
plt.hist(eucl_test, bins=30, alpha=0.7, label="Augmented vs Test", color='#FFB6C1', edgecolor='white')
plt.axvline(x=0.3, color='red', linestyle='--', linewidth=2, label='Threshold')
plt.title("Euclidean Distance Distribution", fontsize=15, fontweight='bold')
plt.xlabel("Euclidean Distance", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(title='Dataset Comparison')
plt.grid(False)
plt.tight_layout()
plt.savefig('euclidean_distance_plot.png', dpi=300, bbox_inches='tight')
plt.show()

# === Cosine Distance Plot ===
plt.figure(figsize=(10, 7))
plt.hist(cos_val, bins=30, alpha=0.7, label="Augmented vs Validation", color='#1E88E5', edgecolor='white')
plt.hist(cos_test, bins=30, alpha=0.7, label="Augmented vs Test", color='#FFB6C1', edgecolor='white')
plt.axvline(x=0.1, color='red', linestyle='--', linewidth=2, label='Threshold')
plt.title("Cosine Distance Distribution", fontsize=15, fontweight='bold')
plt.xlabel("Cosine Distance", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(title='Dataset Comparison')
plt.grid(False)
plt.tight_layout()
plt.savefig('cosine_distance_plot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

# === Paths ===
AUG_IMG_DIR = "D:/PAD-UFES/augmented_images_balanced_800"
AUG_META_CSV = "D:/PAD-UFES/augmented_metadata.csv"
AUG_LABELS_CSV = "D:/PAD-UFES/augmented_labels.csv"
VAL_META_CSV = "D:/PAD-UFES/val_metadata.csv"
TEST_META_CSV = "D:/PAD-UFES/test_metadata.csv"

# === Load metadata ===
aug_meta_df = pd.read_csv(AUG_META_CSV)
aug_labels_df = pd.read_csv(AUG_LABELS_CSV)
val_meta_df = pd.read_csv(VAL_META_CSV)
test_meta_df = pd.read_csv(TEST_META_CSV)

# === Convert to arrays ===
X_aug = aug_meta_df.values
X_val = val_meta_df.values
X_test = test_meta_df.values

# === Compute distances ===
eucl_val_dists = euclidean_distances(X_aug, X_val).min(axis=1)
eucl_test_dists = euclidean_distances(X_aug, X_test).min(axis=1)

cos_val_dists = 1 - cosine_similarity(X_aug, X_val).max(axis=1)
cos_test_dists = 1 - cosine_similarity(X_aug, X_test).max(axis=1)

# === Define thresholds ===
eucl_thresh = 0.3
cos_thresh = 0.1

# === Get indices to delete ===
to_delete_mask = (
    (eucl_val_dists < eucl_thresh) | (eucl_test_dists < eucl_thresh) |
    (cos_val_dists < cos_thresh) | (cos_test_dists < cos_thresh)
)
to_delete_indices = np.where(to_delete_mask)[0]

print(f"\n🧹 Removing {len(to_delete_indices)} augmented samples due to metadata similarity...")

# === Delete image files ===
img_paths_to_delete = aug_labels_df.iloc[to_delete_indices]['ImagePath'].tolist()

for path in img_paths_to_delete:
    try:
        os.remove(path)
    except:
        print(f"⚠️ Couldn't delete image: {path}")

# === Keep only safe samples ===
aug_meta_df_clean = aug_meta_df.drop(index=to_delete_indices).reset_index(drop=True)
aug_labels_df_clean = aug_labels_df.drop(index=to_delete_indices).reset_index(drop=True)

# === Save cleaned files ===
aug_meta_df_clean.to_csv(AUG_META_CSV, index=False)
aug_labels_df_clean.to_csv(AUG_LABELS_CSV, index=False)

print("✅ Metadata + image cleanup complete. Cleaned CSVs saved.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

# === Load metadata ===
aug_meta_df = pd.read_csv("D:/PAD-UFES/augmented_metadata.csv")
val_meta_df = pd.read_csv("D:/PAD-UFES/val_metadata.csv")
test_meta_df = pd.read_csv("D:/PAD-UFES/test_metadata.csv")

# === Convert to arrays ===
X_aug = aug_meta_df.values
X_val = val_meta_df.values
X_test = test_meta_df.values

# === Euclidean Distance ===
eucl_val_matrix = euclidean_distances(X_aug, X_val)
eucl_test_matrix = euclidean_distances(X_aug, X_test)
eucl_val_dists = eucl_val_matrix.min(axis=1)
eucl_test_dists = eucl_test_matrix.min(axis=1)

# === Cosine Distance ===
cos_val_sim = cosine_similarity(X_aug, X_val)
cos_test_sim = cosine_similarity(X_aug, X_test)
cos_val_dists = 1 - cos_val_sim.max(axis=1)
cos_test_dists = 1 - cos_test_sim.max(axis=1)

# === Summary function ===
def summarize_distances(name, distances, threshold):
    return {
        'Type': name,
        'Mean': round(np.mean(distances), 4),
        'Min': round(np.min(distances), 4),
        'Max': round(np.max(distances), 4),
        'Samples Below Threshold': int(np.sum(distances < threshold))
    }

# === Thresholds ===
eucl_thresh = 0.3
cos_thresh = 0.1

# === Collect statistics ===
summary_data = [
    summarize_distances("Euclidean (Aug vs Val)", eucl_val_dists, eucl_thresh),
    summarize_distances("Euclidean (Aug vs Test)", eucl_test_dists, eucl_thresh),
    summarize_distances("Cosine (Aug vs Val)", cos_val_dists, cos_thresh),
    summarize_distances("Cosine (Aug vs Test)", cos_test_dists, cos_thresh),
]

# === Create and display DataFrame ===
summary_df = pd.DataFrame(summary_data)
print("\n📊 Distance Summary Table:")
print(summary_df)

In [ ]:
import os
import pandas as pd
from PIL import Image
import imagehash
from tqdm import tqdm

# === Paths ===
val_paths = X_val_img_paths.tolist()
test_paths = X_test_img_paths.tolist()
train_aug_paths = X_train_img_paths_final.tolist()  # original + augmented

# === Hash all val/test images ===
def compute_hashes(img_paths, label="set"):
    hashes = {}
    for path in tqdm(img_paths, desc=f"Hashing {label}"):
        try:
            img = Image.open(path).resize((224, 224)).convert("RGB")
            img_hash = imagehash.phash(img)
            hashes[path] = img_hash
        except:
            print(f"⚠️ Failed to load {path}")
    return hashes

val_hashes = compute_hashes(val_paths, label="validation")
test_hashes = compute_hashes(test_paths, label="test")
train_hashes = compute_hashes(train_aug_paths, label="train+augmented")

# === Compare hashes for leakage ===
def check_leakage(query_hashes, reference_hashes, threshold=5):
    leaked = []
    for q_path, q_hash in tqdm(query_hashes.items(), desc="Checking leakage"):
        for r_path, r_hash in reference_hashes.items():
            if q_hash - r_hash <= threshold:
                leaked.append((q_path, r_path, q_hash - r_hash))
                break
    return leaked

# === Run leakage detection ===
print("\n🔍 Checking for data leakage in validation set...")
val_leaks = check_leakage(val_hashes, train_hashes)

print("\n🔍 Checking for data leakage in test set...")
test_leaks = check_leakage(test_hashes, train_hashes)

# === Summary ===
print(f"\n🚨 Potential leaks in validation set: {len(val_leaks)}")
print(f"🚨 Potential leaks in test set: {len(test_leaks)}")

if val_leaks:
    print("\nSome examples from validation set leakage:")
    for leak in val_leaks[:3]:
        print(f" - Val: {leak[0]}  ↔  Train: {leak[1]}  [Dist={leak[2]}]")

if test_leaks:
    print("\nSome examples from test set leakage:")
    for leak in test_leaks[:3]:
        print(f" - Test: {leak[0]}  ↔  Train: {leak[1]}  [Dist={leak[2]}]")

In [ ]:
import imagehash
from PIL import Image

# === Compute perceptual hashes ===
def compute_hash_dict(paths):
    hash_dict = {}
    for path in paths:
        try:
            img = Image.open(path).resize((224, 224)).convert('RGB')
            img_hash = imagehash.phash(img)
            hash_dict[path] = img_hash
        except:
            continue
    return hash_dict

# Compute hashes for all sets
train_hashes = compute_hash_dict(X_train_img_paths)
val_hashes = compute_hash_dict(X_val_img_paths)
test_hashes = compute_hash_dict(X_test_img_paths)

# === Compare hashes and flag train samples to remove ===
hash_threshold = 5
leak_train_indices = set()

# Compare against validation set
for val_path, val_hash in val_hashes.items():
    for i, train_path in enumerate(X_train_img_paths):
        if train_path in train_hashes:
            dist = val_hash - train_hashes[train_path]
            if dist <= hash_threshold:
                leak_train_indices.add(i)

# Compare against test set
for test_path, test_hash in test_hashes.items():
    for i, train_path in enumerate(X_train_img_paths):
        if train_path in train_hashes:
            dist = test_hash - train_hashes[train_path]
            if dist <= hash_threshold:
                leak_train_indices.add(i)

print(f"\n🔍 Found {len(leak_train_indices)} potential leakage samples in train set.")

# === Remove leaked train samples ===
mask = ~X_train_img_paths.index.isin(leak_train_indices)
X_train_img_paths = X_train_img_paths[mask].reset_index(drop=True)
X_train_meta = X_train_meta[mask].reset_index(drop=True)
y_train = y_train[mask].reset_index(drop=True)

print(f"✅ Train set cleaned. New size: {len(X_train_img_paths)}")

In [ ]:
import numpy as np

hash_array = np.array(hash_distances)
print("📊 Hamming Distance Summary:")
print(f"Mean:    {np.mean(hash_array):.2f}")
print(f"Std Dev: {np.std(hash_array):.2f}")
print(f"Min:     {np.min(hash_array)}")
print(f"Max:     {np.max(hash_array)}")

In [ ]:
from scipy.spatial.distance import pdist

tsne_dists = pdist(X_embedded, metric='euclidean')
print("📊 t-SNE Embedding Pairwise Distance Summary:")
print(f"Mean:    {np.mean(tsne_dists):.2f}")
print(f"Std Dev: {np.std(tsne_dists):.2f}")
print(f"Min:     {np.min(tsne_dists):.2f}")
print(f"Max:     {np.max(tsne_dists):.2f}")

In [ ]:
df_embedded = pd.DataFrame(X_embedded, columns=['tsne_1', 'tsne_2'])
df_embedded['label'] = aug_labels.values

centroids = df_embedded.groupby('label')[['tsne_1', 'tsne_2']].mean().values
centroid_dists = pdist(centroids, metric='euclidean')

print("📊 Class Centroid Distances in t-SNE space:")
print(f"Mean distance between class centroids: {np.mean(centroid_dists):.2f}")